In [19]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()
sentences = [ "i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)

#paramter
n_step = 2
n_hidden = 5

def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        input = [word_dict[n] for n in sen.split()[:-1]]
        target = word_dict[sen.split()[-1]]
        
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])
    return input_batch, target_batch

In [7]:
X = tf.placeholder(tf.float32, shape=[None, n_step, n_class])
Y = tf.placeholder(tf.float32, shape=[None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
#outputs [batch_size, n_step, n_hidden]

outputs = tf.transpose(outputs, [1,0,2]) #[n_step, batch_size, n_hidden]

outputs = outputs[-1] #[batch_size, n_hidden]
model = tf.matmul(outputs, W)+b  #model  [batch_size, n_class]

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
prediction = tf.cast(tf.argmax(model,1), tf.int32)

In [17]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

input_batch, target_batch = make_batch(sentences)

for epoch in range(5000):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    if (epoch +1) %1000 ==0:
        print("Epoch: {:4d},  cost={:.5f}".format(epoch+1, loss))

In [28]:
input = [sen.split()[:2] for sen in sentences]

predict = sess.run([prediction], feed_dict={X: input_batch})

print(input,'->', [number_dict[n] for n in predict[0]])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
